In [1]:
%%writefile train_logistic.py
import argparse
import pandas as pd 
import numpy as np 
import mltable
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import mlflow
from azure.ai.ml import command 
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from sklearn.preprocessing import LabelEncoder
import pickle
LE=LabelEncoder()

from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

mlflow.autolog()
parser=argparse.ArgumentParser()
parser.add_argument('--training-data', dest='train')
args=parser.parse_args()

df=pd.read_csv(args.train)
df.dropna(inplace=True)
y=df[['Diabetic']]
x=df[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']]
y=LE.fit_transform(y)
X_Train,X_test, y_train,y_test=train_test_split(x,y, test_size=0.30)

model=LogisticRegression(C=0.01)
model.fit(X_Train,y_train)

    #calculate and log accuracy
y_hat=model.predict(X_test)

from sklearn.metrics import accuracy_score
acc=accuracy_score(y_hat, y_test)





Writing train_logistic.py


In [3]:
df.columns

Index(['PatientID', 'Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age',
       'Diabetic'],
      dtype='object')

In [4]:
y

array([0, 0, 0, ..., 1, 0, 0])

In [4]:
from azure.ai.ml import command
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
ws_config={
    "subscription_id": "7b1b43ca-4b64-43cf-9446-edb35a04d7d1",
    "resource_group": "amlws01",
    "workspace_name": "amlwso1"
}

ml_client=MLClient( DefaultAzureCredential(),
                   ws_config['subscription_id'],
                   ws_config['resource_group'],
                   ws_config["workspace_name"])


In [4]:
from azure.ai.ml import command
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
ws_config={
    "subscription_id": "7b1b43ca-4b64-43cf-9446-edb35a04d7d1",
    "resource_group": "amlws01",
    "workspace_name": "amlwso1"
}

ml_client=MLClient( DefaultAzureCredential(),
                   ws_config['subscription_id'],
                   ws_config['resource_group'],
                   ws_config["workspace_name"])


job=command(
    code="./",
    command="python train_logistic.py --training-data diabetes.csv",
    environment='customm-env-scikit:4',
    compute='CE123'
)
ml_client.jobs.create_or_update(job)


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading mlflow-standalone (0.52 MBs)

TypeError: 'str' object is not callable

In [5]:
model=ml_client.models.get('mlflow-logistic-reg', version=1)

In [6]:
#create endpoint
endpoint_name='mlflow-logistic-endpoint'
from azure.ai.ml.entities import BatchEndpoint
endpoint=BatchEndpoint(
    name=endpoint_name,
    description='logistic-reg-batch-ep'
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

In [7]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction
deployment=BatchDeployment(
    name='mlflow-logistic-deployed',
    description='logistic-regression-classifier',
    endpoint_name=endpoint_name,
    model=model,
    compute='CE123',
    instance_count=1,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name='mlflow-predictions.csv',
    retry_settings=BatchRetrySettings(max_retries=3, timeout=3600),
    logging_level='info'
)

ml_client.batch_deployments.begin_create_or_update(deployment)

In [9]:
endpoint.defaults={}
endpoint.defaults['deployment_name']=deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint)

In [14]:
unlabelled_data=ml_client.data.get(name='patient-data-unlabeled', version=1)


from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input=Input(types=AssetTypes.URI_FOLDER, path=unlabelled_data.id)
job=ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=input
)

ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
mlflow-logistic-endpoint,batchjob-e84aa8c9-c48d-46ca-9028-bb3a916d585a,pipeline,Preparing,Link to Azure Machine Learning studio


In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential=DefaultAzureCredential()
    credential.get_token('https://management.azure.com./default')
except Exception as ex:
    credential=InteractiveBrowserCredential()

ml_client=MLClient.from_config(credential=credential)



DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: Unexpected content type "text/plain; charset=utf-8"
Content: SSO failure: unable to retrieve a token. Error Token service ce123. Token not found
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.
Found the config file in: /config.json


In [5]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
endpoint_name='managed-mlflow-online-endpoint'
endpoint=ManagedOnlineEndpoint(
    name=endpoint_name,
    description='Real time endpoint inferencing for MLFlow model',
    auth_mode="key"
)

ml_client.begin_create_or_update(endpoint)

In [6]:
#model registration
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

from azure.ai.ml.entities import ManagedOnlineDeployment
model=ml_client.models.get('mlflow-logistic-reg', version=1)

In [8]:
blue_deployment=ManagedOnlineDeployment(
    name='blue',
    endpoint_name=endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint managed-mlflow-online-endpoint exists


.........

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'managed-mlflow-online-endpoint', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/odidp:c175b917-3947-4201-847b-196c4048528c:525097b6-5f5e-47f7-9784-342fd178c87e?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/resourceGroups/amlws01/providers/Microsoft.MachineLearningServices/workspaces/amlwso1/onlineEndpoints/managed-mlflow-online-endpoint/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ce123/code/Users/satyakebakshi95/mlflow-standalone', 'creation_context': None, 'serialize': <msrest.serializat

In [9]:
#blue green deployment
endpoint.traffic={"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://managed-mlflow-online-endpoint.northcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://managed-mlflow-online-endpoint.northcentralus.inference.ml.azure.com/swagger.json', 'name': 'managed-mlflow-online-endpoint', 'description': 'Real time endpoint inferencing for MLFlow model', 'tags': {}, 'properties': {'createdBy': 'Satyake Bakshi', 'createdAt': '2024-07-31T13:23:40.907862+0000', 'lastModifiedAt': '2024-07-31T13:23:40.907862+0000', 'azureml.onlineendpointid': '/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/resourcegroups/amlws01/providers/microsoft.machinelearningservices/workspaces/amlwso1/onlineendpoints/managed-mlflow-online-endpoint', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/oeidp:c175b917-394

In [11]:
%%writefile sample_data.json
{
  "input_data": {
    "columns": [
      "Pregnancies", 
      "PlasmaGlucose", 
      "DiastolicBloodPressure",
      "TricepsThickness",
      "SerumInsulin",
      "BMI", 
      "DiabetesPedigree",
      "Age"
    ],
    "index": [1],
    "data": [
      [0, 148, 18,89, 179, 39.19, 0.18, 45
      ]
    ]
  }
}




Overwriting sample_data.json


In [12]:
response=ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment="blue",
    request_file="sample_data.json"
)

if response[1]==1:
    print("Diabetic")
else:
    print("Not diabetic")

Not diabetic
